In [1]:
!pip install boto3

In [2]:
import os
import boto3
import json
import requests

# Utilitary build functions

In [3]:
def build_ecr_password_stdin(account_id_, region_):
    return f"{account_id_}.dkr.ecr.{region_}.amazonaws.com"

def build_tagged_image(image_name, tag):
    return f"{image_name}:{tag}"

def build_ecr_url(account_id_, region_, image_name, tag):    
    tagged_image_name=build_tagged_image(image_name, tag)
    password_stdin=build_ecr_password_stdin(account_id_, region_)
    
    return f"{password_stdin}/{tagged_image_name}"

def build_lambda_uri(region_, lambda_arn_):
    uri_host=f"arn:aws:apigateway:{region_}:lambda:path"
    uri_route=f"2015-03-31/functions/{lambda_arn_}/invocations"
    return f"{uri_host}/{uri_route}"

def build_source_arn(region_, account_id_, rest_api_id_):
    return f'arn:aws:execute-api:{region_}:{account_id_}:{rest_api_id_}/*'

def build_api_url(rest_api_id, region_, endpoint_, stage_):
    host=f"https://{rest_api_id}.execute-api.{region_}.amazonaws.com"
    route=f"{stage_}/{endpoint_}/"
    return f"{host}/{route}"


# Information for communication protocols

In [4]:
# Account
account_id = '060004687794'

# Server
region = "sa-east-1"

# Platform
ecr_image_name = "serverless-example"
tag='latest'
target_platform="linux/amd64"

# API
endpoint = "predict"
method_verb='POST'
stage = "test"

# Ellaborate information
tagged_image_uri=f"{ecr_image_name}:latest"
password_stdin=f"{account_id}.dkr.ecr.{region}.amazonaws.com"

# Specify the role name and trust policy for the Lambda service
role_name = 'lambda-exec-role'

trust_policy = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {'Service': 'lambda.amazonaws.com'},
            'Action': 'sts:AssumeRole'
        }
    ]
}

policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'

# Rate limits: Harsh since this will be public facing
# Quota: Low daily limits for the same reason
usage_constraints = {
    'rate_limits': {
        'burstLimit': 10,
        'rateLimit': 10.0
    },
    'quota': {
        'limit': 100,
        'period': 'DAY'
    }
}

# Function name (not public facing)
function_name = f'lambda-fn-{ecr_image_name}'

# Clients

In [5]:
# Set up the IAM client
iam_client = boto3.client('iam', region_name=region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=region)

# Development steps

- IAM role image handling;
- Docker image
- Lambda function creation;
- API Gateway

## IAM Role

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 



## Utilitary functions

In [6]:
def try_get_role(i_client, role_name_, trust_policy):
    try:
        return i_client.get_role(
            RoleName=role_name_
        )
    except i_client.exceptions.NoSuchEntityException:
        i_client.create_role(
            RoleName=role_name_,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Execution role for Lambda function',
        )
        
def try_attach_role_policy(i_client, role_name_, policy_arn, trust_policy):
    # Just need to run it once, otherwise retrieve already existing role
    response=try_get_role(i_client, role_name_, trust_policy)

    # Get the role ARN
    role_arn = response['Role']['Arn']

    # Attach the AWSLambdaBasicExecutionRole policy to the role
    i_client.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
    
    return role_arn

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, role_name, policy_arn, trust_policy)

In [7]:
role_arn

'arn:aws:iam::060004687794:role/lambda-exec-role'

## Docker image

Run 2 cells below with key stroke _Shift+Enter_ to upload the docker image to ECR.

In [8]:
def run(command):
    os.system(command)

def login_ecr_docker(account_id, region):
    password_stdin=build_ecr_password_stdin(account_id, region)
    
    opts=f"get-login-password --region {region}"
    get_pwd_command=f"aws ecr {opts}"
    
    opts=f"--username AWS --password-stdin {password_stdin}"
    login_command=f"docker login {opts}"
    
    entry_command=f"{get_pwd_command} | {login_command}"
    
    run(entry_command)
    
def create_ecr_image(ecr_image_name_):
    args_1=f"--repository-name {ecr_image_name_}"
    args_2=f"--image-scanning-configuration scanOnPush=true"
    args_3=f"--image-tag-mutability MUTABLE"
    create_args=f"{args_1} {args_2} {args_3}"
    
    opts=f"create-repository {create_args}"
    create_comand=f"aws ecr {opts}"

    run(create_comand)

def delete_ecr_image(ecr_image_name_):
    tags=f"--force --repository-name {ecr_image_name_}"
    delete_command=f"aws ecr delete-repository {tags}" 
    
    run(delete_command)
    
def build_docker_image(ecr_image_name):
    build_args=f"-t {ecr_image_name} ."
    build_command=f"docker build {build_args}"
    
    run(build_command)
    
def tag_docker_image(tagged_image_uri_, routed_url):
    tag_args=f"{tagged_image_uri_} {routed_url}"
    tag_command=f"docker tag {tag_args}"
    
    run(tag_command)
    
def push_docker_image(tagged_image_uri):
    push_command=f"docker push {tagged_image_uri}"

    run(push_command)
    
def pipe_push_image(account_id_, region_, ecr_image_name_, tag_):
    # 1. Log in to AWS ECR
    login_ecr_docker(account_id_, region_)
    
    # 2. Delete ECR repo: only needs to be done once
    delete_ecr_image(ecr_image_name_)
    
    # 3. Create ECR repo: only needs to be done once
    create_ecr_image(ecr_image_name_)

    # 4. Build Docker image using your local Dockerfile
    build_docker_image(ecr_image_name_)

    # 5. Tag you image
    tagged_image_uri=build_tagged_image(ecr_image_name_, tag_)
    routed_url=build_ecr_url(account_id_, region_, ecr_image_name_, tag_)
    tag_docker_image(tagged_image_uri, routed_url)

    # 6. Push your image to ECR
    push_docker_image(routed_url)

In [10]:
pipe_push_image(account_id, region, ecr_image_name, tag)

WARNING! Your password will be stored unencrypted in /home/brunolnetto/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded



An error occurred (RepositoryNotFoundException) when calling the DeleteRepository operation: The repository with name 'serverless-example' does not exist in the registry with id '060004687794'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:sa-east-1:060004687794:repository/serverless-example",
        "registryId": "060004687794",
        "repositoryName": "serverless-example",
        "repositoryUri": "060004687794.dkr.ecr.sa-east-1.amazonaws.com/serverless-example",
        "createdAt": "2023-09-15T20:33:08-03:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 580B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for public.ecr.aws/lambda/python:3.9.2023.03.15.15-x86_64
#3 DONE 1.7s

#4 [1/4] FROM public.ecr.aws/lambda/python:3.9.2023.03.15.15-x86_64@sha256:d7dd8369b33bb0825b32269af61cab13e4c5db6200ad6350b6aee0c4682b5562
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 225.47kB 0.0s done
#5 DONE 0.0s

#4 [1/4] FROM public.ecr.aws/lambda/python:3.9.2023.03.15.15-x86_64@sha256:d7dd8369b33bb0825b32269af61cab13e4c5db6200ad6350b6aee0c4682b5562
#4 CACHED

#6 [2/4] COPY requirements.txt .
#6 DONE 0.1s

#7 [3/4] RUN pip3 install -r requirements.txt --target "/var/task"
#7 0.595 Ignoring colorama: markers 'python_version >= "3.8" and python_version < "4.0" and sys_platform == "win32"' don't match your environment
#7 0.900 Collecting autopep8==2.0.4
#7 1.001   Downloading a

The push refers to repository [060004687794.dkr.ecr.sa-east-1.amazonaws.com/serverless-example]
2d51df1dbc09: Preparing
80b68e8b9149: Preparing
60fb27e38b97: Preparing
2d32c89b4065: Preparing
e5a063df86fb: Preparing
2d244e0816c6: Preparing
ef5cc9241c56: Preparing
65c2fe7d7702: Preparing
0d91c86bcea1: Preparing
ef5cc9241c56: Waiting
65c2fe7d7702: Waiting
2d244e0816c6: Waiting
0d91c86bcea1: Waiting
60fb27e38b97: Pushed
2d244e0816c6: Pushed
ef5cc9241c56: Pushed
2d51df1dbc09: Pushed
2d32c89b4065: Pushed
80b68e8b9149: Pushed
65c2fe7d7702: Pushed
e5a063df86fb: Pushed
0d91c86bcea1: Pushed
latest: digest: sha256:916a79637ade9e862bd96de7c594a9c62cbeb5ebddd06abff24fc49214ece130 size: 2212


# Lambda function 

Run 2 cells below with key stroke _Shift+Enter_ to generate a Lambda Function based on Docker Image

In [11]:
def get_function(l_client, function_name):
    failure_message=f"Lambda function {function_name} does not exist"
    
    try:
        return l_client.get_function(FunctionName=function_name)
    
    except l_client.exceptions.ResourceNotFoundException:
        print(failure_message)

def create_function(l_client, function_name, routed_url, role_arn):
    failure_message=f"Lambda function {function_name} already exists"
    
    code_payload={'ImageUri': routed_url}
    func_description='SKLearn predict Lambda function'
    
    try:
        return l_client.create_function(
            FunctionName=function_name,
            Role=role_arn,
            PackageType='Image',
            Code=code_payload,
            Description=func_description,
            Timeout=10,
            MemorySize=256,
            Publish=True,
        )
    
    except l_client.exceptions.ResourceNotFoundException:
        print(failure_message)

# snippet-start:[python.example_code.lambda.DeleteFunction]
def delete_function(l_client, function_name):
    """
    Deletes a Lambda function.

    :param function_name: The name of the function to delete.
    """
    try:
        return l_client.delete_function(FunctionName=function_name)
    except l_client.exceptions.ClientError:
        print(f"Couldn't delete function {function_name}.", )

### Create function

In [12]:
# Retrieve (if already exists) or create a new Lambda function
routed_url=build_ecr_url(account_id, region, ecr_image_name, tag)
deletion_response = delete_function(lambda_client, function_name)
create_response = create_function(lambda_client, function_name, routed_url, role_arn)

create_response

Couldn't delete function lambda-fn-serverless-example.


{'ResponseMetadata': {'RequestId': 'fbe94f7f-aebd-429d-837e-72784684061d',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Fri, 15 Sep 2023 23:33:54 GMT',
   'content-type': 'application/json',
   'content-length': '1111',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fbe94f7f-aebd-429d-837e-72784684061d'},
  'RetryAttempts': 0},
 'FunctionName': 'lambda-fn-serverless-example',
 'FunctionArn': 'arn:aws:lambda:sa-east-1:060004687794:function:lambda-fn-serverless-example',
 'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
 'CodeSize': 0,
 'Description': 'SKLearn predict Lambda function',
 'Timeout': 10,
 'MemorySize': 256,
 'LastModified': '2023-09-15T23:33:53.534+0000',
 'CodeSha256': '916a79637ade9e862bd96de7c594a9c62cbeb5ebddd06abff24fc49214ece130',
 'Version': '20',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '4c3df056-4676-4d60-bab7-c87056d572be',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode': 'Creating

In [16]:
get_response = get_function(lambda_client, function_name)

get_response

{'ResponseMetadata': {'RequestId': '0dbe22dd-b08d-46f6-af4f-5e6d2486a6d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Sep 2023 23:34:10 GMT',
   'content-type': 'application/json',
   'content-length': '1427',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0dbe22dd-b08d-46f6-af4f-5e6d2486a6d8'},
  'RetryAttempts': 0},
 'Configuration': {'FunctionName': 'lambda-fn-serverless-example',
  'FunctionArn': 'arn:aws:lambda:sa-east-1:060004687794:function:lambda-fn-serverless-example',
  'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
  'CodeSize': 0,
  'Description': 'SKLearn predict Lambda function',
  'Timeout': 10,
  'MemorySize': 256,
  'LastModified': '2023-09-15T23:33:53.534+0000',
  'CodeSha256': '916a79637ade9e862bd96de7c594a9c62cbeb5ebddd06abff24fc49214ece130',
  'Version': '$LATEST',
  'TracingConfig': {'Mode': 'PassThrough'},
  'RevisionId': 'd729d8ae-cb5c-4076-a04a-0ad32e62f09c',
  'State': 'Active',
  'LastUpdateStatus': 'Successful',
  'Packag

### Test function

In [30]:
from lambda_api.predict_service import validate_event
from json import loads

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
event={"body": example}
context={}

payload = validate_event(event, context)

loads(payload['body'])

[1, 2, 3, 4, 5]

'{"body": [1, 2, 3, 4, 5]}'

In [17]:
# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = json.loads(response['Payload'].read())

print(result)

{'statusCode': 500, 'headers': {'Content-Type': 'application/json'}, 'body': '[]', 'isBase64Encoded': False, 'error_message': "unsupported operand type(s) for ** or pow(): 'str' and 'int'"}


## API Gateway setup

Run the cells below to set the Lambda Function as an Endpoint on API Gateway.

In [26]:
def has_api(g_client, rest_api_name_):
    response = g_client.get_rest_apis()
    create_api_on_gateway = True
    
    for item in response['items']:
        if item['name'] == rest_api_name_:
            create_api_on_gateway = False
            
    return create_api_on_gateway

def create_resource(g_client, rest_api_id_):
    response = g_client.get_resources(restApiId=rest_api_id_)
    root_id = response['items'][0]['id']
    
    response = gateway_client.create_resource(
        restApiId=rest_api_id_,
        parentId=root_id,
        pathPart=endpoint,
    )
    
    resource_id = response['id']
    
    return resource_id

def create_rest_method(g_client, rest_api_id_, resource_id_, method_verb_):
    g_client.put_method(
        restApiId=rest_api_id_,
        resourceId=resource_id_,
        httpMethod=method_verb_,
        authorizationType='NONE', # WARNING: this will allow public access!
        apiKeyRequired=True,
    )

def create_rest_api(g_client, rest_api_name_):
    description='API Gateway that triggers a lambda function'
    response=g_client.create_rest_api(name=rest_api_name_, description=description) 
    
    rest_api_id = response['id']
    
    return rest_api_id

def get_lambda_arn(g_client, function_name_):
    response = g_client.get_function(FunctionName=function_name_)
    return response['Configuration']['FunctionArn']

def setup_integration(g_client, lambda_uri_, rest_api_id_, resource_id_, method_verb_):
    g_client.put_integration(
        restApiId=rest_api_id_,
        resourceId=resource_id_,
        httpMethod=method_verb_,
        type='AWS_PROXY',
        integrationHttpMethod=method_verb_,
        uri=lambda_uri_,
    )
    
def create_deployment(g_client, rest_api_id_, stage_):
    g_client.create_deployment(restApiId=rest_api_id_, stageName=stage_)
    
def create_api_key(g_client, rest_api_name_):
    response = g_client.create_api_key(
        name=rest_api_name_ + '-key',
        description='API key',
        enabled=True,
        generateDistinctId=True
    )
    
    api_key_id = response['id']
    api_key_value = response['value']
    
    return api_key_id, api_key_value

def create_usage_plan(g_client, rest_api_id_, stage_, usage_constraints_):
    name='API usage plan'
    description='Harsh rate limits and daily quota for public facing API'
    stages=[
        {
            'apiId': rest_api_id_,
            'stage': stage_,
        },
    ]
    
    response = g_client.create_usage_plan(
        name=name,
        description=description,
        apiStages=stages,
        throttle=usage_constraints_['rate_limits'],
        quota=usage_constraints_['quota']
    )
    
    usage_plan_id = response['id']
    
    return usage_plan_id

def create_usage_plan_key(g_client, usage_plan_id_, api_key_id_):
    g_client.create_usage_plan_key(
        usagePlanId=usage_plan_id_,
        keyId=api_key_id_,
        keyType='API_KEY'
    )
    
def add_apigateway_permission(l_client, function_name_, source_arn_):
    return l_client.add_permission(
        FunctionName=function_name_,
        StatementId='apigateway-lambda-invoke-permission',
        Action='lambda:InvokeFunction',
        Principal='apigateway.amazonaws.com',
        SourceArn=source_arn_
    )

def deploy_rest_api(\
        g_client, l_client, \
        account_id, region, \
        function_name_, rest_api_name_, endpoint_, method_verb_, \
        usage_constraints_, stage_ \
    ):
    # First, lets verify whether we already have an endpoint with this name.
    if not has_api(g_client, rest_api_name_):

        # 1. Create REST API
        rest_api_id = create_rest_api(g_client, rest_api_name_)

        # 2. Create resource
        resource_id=create_resource(g_client, rest_api_id)
        
        # 3. Create method
        create_rest_method(g_client, rest_api_id, resource_id, method_verb_)
        
        # 4. Get the Lambda function ARN
        lambda_arn = get_lambda_arn(l_client, function_name_)

        # 5. Set up integration with the Lambda function
        lambda_uri = build_lambda_uri(region, lambda_arn)

        setup_integration(g_client, lambda_uri, rest_api_id, resource_id, method_verb_)

        # 6. Deploy API
        create_deployment(g_client, rest_api_id, stage_)

        # 7. Create API key
        api_key_id, api_key_value = create_api_key(g_client, rest_api_name_)

        # 8. Create usage plan
        usage_plan_id = create_usage_plan(g_client, rest_api_id, stage_, usage_constraints_)
        
        # 9. Associate the usage plan with the API key
        create_usage_plan_key(g_client, usage_plan_id, api_key_id)
        
        # 10. Grant API Gateway permission to invoke the Lambda function
        source_arn = build_source_arn(region, account_id, rest_api_id)
        
        try: 
            add_apigateway_permission(l_client, function_name_, source_arn)
        except l_client.exceptions.ResourceConflictException:
            pass
        
        return {
            'url': build_api_url(rest_api_id, region, endpoint_, stage_),
            'api_key': api_key_value,
            'usage_plan_id': usage_plan_id,
            'rest_api_id': rest_api_id
        }
    
    else: 
        failure_msg=f"REST API name {rest_api_name_} is already under usage!"
        print(failure_msg)
        
        return {}

In [27]:
# Define the name of the API (not public facing)
rest_api_name = function_name + '-api'

deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    account_id, region, \
    function_name, rest_api_name, endpoint, method_verb, \
    usage_constraints, stage \
)

deployment_reponse

{'url': 'https://qaq2njf3d3.execute-api.sa-east-1.amazonaws.com/test/predict/',
 'api_key': 'wGhDa12k9ran6jrLnWwRB3moywsCrHZzaJkxTYhm',
 'usage_plan_id': 'p3kvll',
 'rest_api_id': 'qaq2njf3d3'}

In [28]:
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, region, endpoint, stage)
print(api_url)


https://qaq2njf3d3.execute-api.sa-east-1.amazonaws.com/test/predict/


In [29]:
api_key=deployment_reponse['api_key']
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, region, endpoint, stage)
print(api_key)
print(api_url)

wGhDa12k9ran6jrLnWwRB3moywsCrHZzaJkxTYhm
https://qaq2njf3d3.execute-api.sa-east-1.amazonaws.com/test/predict/


In [30]:
# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)
resp.json()


{'message': 'Internal server error'}